# Data Processing Notebook Template

Use this scaffold to build repeatable PySpark pipelines with logging, validations, and optional Delta rollbacks. Replace placeholders with your sources, business logic, and checks.


## Notebook guidelines

- Name notebooks in clear snake_case (e.g., `orders_enriched.ipynb` or `domain_orders_enriched.ipynb`); keep one primary table per notebook.
- Make runs idempotent: deterministic transforms, safe overwrites/merges, and repeatable partition logic so reruns do not create duplicates.
- Keep scopes clean: functions in snake_case, classes in PascalCase, constants UPPER_SNAKE, modules/files in snake_case; avoid implicit globals beyond parameters.
- Encapsulate helpers in small functions inside the notebook when reusable; prefer pure functions and pass Spark/DataFrames explicitly.
- Document inputs/outputs near the top and ensure the notebook owns producing one curated table/view, not many.


In [1]:
from spark_fuse.spark import create_session
from spark_fuse.utils import change_tracking  # noqa: F401 (enables .change_tracking accessors)
from spark_fuse.utils.logging import create_progress_tracker, enable_spark_logging, log_progress as log_progress_step, console
from spark_fuse.utils.dataframe import ensure_columns, preview
from pyspark.sql import functions as F, types as T
import datetime as _dt

progress_tracker = create_progress_tracker(total_steps=10)
log = console()

def log_progress(label: str) -> None:
    log_progress_step(progress_tracker, log, label)

# Set any reusable parameters here
job_ts = _dt.datetime.now().replace(microsecond=0).isoformat()  # UTC timestamp; override if needed


> Why `functions as F` and `types as T`? Aliasing keeps chained expressions concise, matches common Spark style, and avoids polluting the global namespace with hundreds of column functions and type classes.


## Create a session

Adjust `app_name`, `master`, and configs for your environment.


In [2]:
log.log("[INFO] Starting Spark session...")
spark = create_session(
    app_name="data-processing-template",
    master="local[*]",
    # extra_configs={"spark.some.credential": "value"},
)
log.log("[INFO] Spark session ready.")
log_progress("Spark session ready")
spark


[15:06:12] [INFO] Starting Spark session...                                                         ]8;id=144883;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1197149869.py\1197149869.py]8;;\:]8;id=399583;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1197149869.py#1\1]8;;\

:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/kevin/.ivy2.5.2/cache
The jars for the packages stored in: /Users/kevin/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d262cd46-4c42-4bec-b7da-a6cac5f2ec89;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 104ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from central in [default]
	io.delta#delta-storage;4.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.13.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules     

[15:06:15] [INFO] Spark session ready.                                                              ]8;id=471241;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1197149869.py\1197149869.py]8;;\:]8;id=726118;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1197149869.py#7\7]8;;\

           [INFO]  1/10 Spark session ready (+3.52s, total 3.52s)                                     ]8;id=555894;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=612032;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Start logging

Raise Spark log verbosity while you iterate so shuffle and scheduler details show up in the driver logs.


In [3]:
enable_spark_logging(spark, level="WARN")
log.log("[INFO] Spark logging enabled at WARN.")
log_progress("Logging configured")


           [INFO] Spark logging enabled at WARN.                                                     ]8;id=650598;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/692621756.py\692621756.py]8;;\:]8;id=970242;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/692621756.py#2\2]8;;\

           [INFO]  2/10 Logging configured (+0.13s, total 3.65s)                                      ]8;id=225469;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=409859;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Load relevant data

Declare input locations and load dataframes. Swap formats and options for your sources.


In [4]:
log.log("[INFO] Loading input data (dummy samples; replace with real sources)...")

orders_schema = T.StructType(
    [
        T.StructField("order_id", T.StringType(), False),
        T.StructField("order_ts", T.StringType(), False),
        T.StructField("order_total", T.DoubleType(), False),
        T.StructField("customer_id", T.StringType(), False),
    ]
)
orders_data = [
    ("O-1001", "2024-01-05", 42.50, "C001"),
    ("O-1002", "2024-01-06", 18.00, "C002"),
    ("O-1003", "2024-01-06", 120.75, "C003"),
]
orders_df = spark.createDataFrame(orders_data, schema=orders_schema)

customers_schema = T.StructType(
    [
        T.StructField("customer_id", T.StringType(), False),
        T.StructField("segment", T.StringType(), True),
        T.StructField("country", T.StringType(), True),
    ]
)
customers_data = [
    ("C001", "retail", "US"),
    ("C002", "enterprise", "CA"),
    ("C003", "retail", "UK"),
]
customers_df = spark.createDataFrame(customers_data, schema=customers_schema)

log.log("[INFO] Input data loaded.")
log_progress("Input data loaded")
log.log(f"[INFO] Orders sample:\n{preview(orders_df)}")


           [INFO] Loading input data (dummy samples; replace with real sources)...                  ]8;id=321896;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1364864045.py\1364864045.py]8;;\:]8;id=931103;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1364864045.py#1\1]8;;\

[15:06:16] [INFO] Input data loaded.                                                               ]8;id=297433;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1364864045.py\1364864045.py]8;;\:]8;id=918156;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1364864045.py#32\32]8;;\

           [INFO]  3/10 Input data loaded (+0.42s, total 4.07s)                                       ]8;id=886097;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=531465;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

[15:06:17] [INFO] Orders sample:                                                                   ]8;id=279842;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1364864045.py\1364864045.py]8;;\:]8;id=991337;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1364864045.py#34\34]8;;\
           rows=[{'order_id': 'O-1001', 'order_ts': '2024-01-05', 'order_total': 42.5,                             
           'customer_id': 'C001'}, {'order_id': 'O-1002', 'order_ts': '2024-01-06', 'order_total':                 
           18.0, 'customer_id': 'C002'}, {'order_id': 'O-1003', 'order_ts': '2024-01-06',                          
           'order_total': 120.75, 'customer_id': 'C003'}]                                                          
           schema=struct<order_id:string,order_ts:string,order_total:double,customer_id:string>                    

## Process data

Apply your business logic: filtering, casting, enrichment, and derived columns.


In [5]:
log.log("[INFO] Curating datasets...")
curated_orders_df = (
    orders_df
    .withColumn("order_date", F.to_date("order_ts"))
    .withColumn("order_month", F.date_format("order_date", "yyyy-MM"))
    .withColumn("processing_ts", F.lit(job_ts))
)

curated_customers_df = customers_df.select("customer_id", "segment", "country")
log.log("[INFO] Curated orders and customers dataframes ready.")
log_progress("Curated datasets ready")


           [INFO] Curating datasets...                                                              ]8;id=9793;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/2402535332.py\2402535332.py]8;;\:]8;id=213847;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/2402535332.py#1\1]8;;\

           [INFO] Curated orders and customers dataframes ready.                                   ]8;id=728387;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/2402535332.py\2402535332.py]8;;\:]8;id=836487;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/2402535332.py#10\10]8;;\

           [INFO]  4/10 Curated datasets ready (+1.56s, total 5.64s)                                  ]8;id=568390;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=26638;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Do joins

Join curated datasets and pick the right join strategy for your domain (inner/left/anti).


In [6]:
log.log("[INFO] Joining curated datasets...")
joined_df = (
    curated_orders_df.alias("o")
    .join(curated_customers_df.alias("c"), on="customer_id", how="left")
)

log.log(f"[INFO] Join complete. Sample:\n{preview(joined_df)}")
log_progress("Join complete")


           [INFO] Joining curated datasets...                                                       ]8;id=122999;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3959396089.py\3959396089.py]8;;\:]8;id=395355;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3959396089.py#1\1]8;;\

[15:06:18] [INFO] Join complete. Sample:                                                            ]8;id=448694;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3959396089.py\3959396089.py]8;;\:]8;id=715241;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3959396089.py#7\7]8;;\
           rows=[{'customer_id': 'C001', 'order_id': 'O-1001', 'order_ts': '2024-01-05',                           
           'order_total': 42.5, 'order_date': datetime.date(2024, 1, 5), 'order_month': '2024-01',                 
           'processing_ts': '2025-12-01T15:06:12', 'segment': 'retail', 'country': 'US'},                          
           {'customer_id': 'C002', 'order_id': 'O-1002', 'order_ts': '2024-01-06', 'order_total':                  
           18.0, 'order_date': datetime.date(2024, 1, 6), 'order_month': '2024-01',                                
           'processing_ts': '2025-12-01T15:06:12', 'segment': 'enterprise', 'country': 'CA'},                      
           {'customer_id': 'C003', 'order_id': 'O-1003', 'order_ts': '2024-01-06', 'order_total':                  
           120.75, 'order_date': datetime.date(2024, 1, 6), 'order_month': '2024-01',                              
           'processing_ts': '2025-12-01T15:06:12', 'segment': 'retail', 'country': 'UK'}]                          
           schema=struct<customer_id:string,order_id:string,order_ts:string,order_total:double,orde                
           r_date:date,order_month:string,processing_ts:string,segment:string,country:string>                      

           [INFO]  5/10 Join complete (+0.79s, total 6.43s)                                           ]8;id=881658;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=245089;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Do data tests

Add lightweight checks so issues surface early during development.


In [7]:
log.log("[INFO] Running in-memory data tests...")
# Schema/column guardrails
ensure_columns(joined_df, ["order_id", "customer_id", "order_date"])

# Null/uniqueness/data quality checks (expand as needed)
assert joined_df.filter(F.col("order_id").isNull()).count() == 0, "order_id should be populated"
assert joined_df.filter(F.col("customer_id").isNull()).count() == 0, "customer_id should be populated"
assert joined_df.dropDuplicates(["order_id"]).count() == joined_df.count(), "order_id should be unique"

# Domain-specific rule example; swap column names for your metric
invalid_states = joined_df.filter(F.col("order_total") < 0).count()
assert invalid_states == 0, f"Found {invalid_states} negative order totals"
log.log("[INFO] In-memory data tests passed.")
log_progress("In-memory data tests passed")


           [INFO] Running in-memory data tests...                                                   ]8;id=875377;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1438265954.py\1438265954.py]8;;\:]8;id=505615;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1438265954.py#1\1]8;;\

[15:06:19] [INFO] In-memory data tests passed.                                                     ]8;id=569582;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1438265954.py\1438265954.py]8;;\:]8;id=628062;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/1438265954.py#13\13]8;;\

           [INFO]  6/10 In-memory data tests passed (+1.32s, total 7.75s)                             ]8;id=191799;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=604552;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Write data & post-write tests

Persist curated results, run post-write validations, and attempt rollback when Delta Lake is available.


In [8]:
output_path = "/tmp/spark_fuse/orders_enriched_ct"  # dummy local path; replace with real target (e.g., s3://bucket/silver/orders)
target_table = "orders_enriched_ct"  # metastore table name if you want one registered
log.log(f"[INFO] Preparing to write dataset to {output_path}")

delta_supported = False
pre_write_version = None
output_format = "delta"
table_exists = False
try:
    from delta.tables import DeltaTable
    delta_supported = True
    try:
        DeltaTable.forPath(spark, output_path)
        table_exists = True
        log.log("[INFO] Existing Delta table found; skipping DDL.")
    except Exception:
        log.log("[INFO] No existing Delta table found at output path; creating with change-tracking columns.")
        (
            DeltaTable.createIfNotExists(spark)
            .tableName(target_table)
            .location(output_path)
            .addColumn("order_id", T.StringType())
            .addColumn("order_ts", T.StringType())
            .addColumn("order_total", T.DoubleType())
            .addColumn("customer_id", T.StringType())
            .addColumn("order_date", T.DateType())
            .addColumn("order_month", T.StringType())
            .addColumn("processing_ts", T.StringType())
            .addColumn("segment", T.StringType())
            .addColumn("country", T.StringType())
            .addColumn("effective_start_ts", T.TimestampType())
            .addColumn("effective_end_ts", T.TimestampType())
            .addColumn("is_current", T.BooleanType())
            .addColumn("version", T.LongType())
            .addColumn("row_hash", T.StringType())
            .addColumn("load_ts", T.TimestampType())
            .execute()
        )
except Exception:
    output_format = "parquet"
    log.log("[INFO] Delta Lake not available; falling back to parquet for write and disabling rollback.")

log.log(f"[INFO] Writing data to final path with format={output_format}...")
log_progress("Write prerequisites complete")

if delta_supported:
    change_tracking_options = {
        "business_keys": ["order_id"],
        "tracked_columns": [
            "order_id",
            "customer_id",
            "order_date",
            "order_month",
            "processing_ts",
            "segment",
            "country",
            "order_total",
        ],
        "load_ts_expr": "current_timestamp()",
        "create_if_not_exists": not table_exists,
        "allow_schema_evolution": True,
    }
    joined_df.write.change_tracking.options(
        change_tracking_mode="track_history",
        change_tracking_options=change_tracking_options,
    ).table(output_path)
else:
    (
        joined_df.write
        .option("mergeSchema", "true")
        .mode("overwrite")
        .format(output_format)
        .partitionBy("order_month")
        .save(output_path)
    )

log_progress("Write complete")

log.log("[INFO] Running post-write validations on persisted data...")
persisted_df = spark.read.format(output_format).load(output_path)
current_df = persisted_df.filter(F.col("is_current") == F.lit(True)) if delta_supported else persisted_df

try:
    ensure_columns(current_df, ["order_id", "customer_id", "order_date", "order_month"])
    assert current_df.count() > 0, "Persisted dataset is empty"
    assert current_df.filter(F.col("order_id").isNull()).count() == 0, "order_id should be populated"
    assert current_df.dropDuplicates(["order_id"]).count() == current_df.count(), "order_id should be unique"
    invalid_persisted_states = current_df.filter(F.col("order_total") < 0).count()
    assert invalid_persisted_states == 0, f"Found {invalid_persisted_states} negative order totals after write"
    log.log("[INFO] Post-write validations passed.")
    log_progress("Post-write validations passed")
except Exception as exc:
    log.log(f"[ERROR]Post-write validation failed: {exc}")
    if delta_supported and pre_write_version is not None:
        try:
            log.log(f"[INFO] Attempting Delta rollback to version {pre_write_version} ...")
            spark.sql(f"RESTORE TABLE delta.`{output_path}` TO VERSION AS OF {pre_write_version}")
            log.log("[INFO] Rollback succeeded.")
        except Exception as rollback_exc:
            log.log(f"[ERROR]Rollback attempt failed: {rollback_exc}")
    else:
        log.log("[INFO] No rollback available; inspect persisted data manually.")
    raise


           [INFO] Preparing to write dataset to /tmp/spark_fuse/orders_enriched_ct                  ]8;id=635637;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3951893949.py\3951893949.py]8;;\:]8;id=61396;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3951893949.py#3\3]8;;\

[15:06:20] [INFO] Existing Delta table found; skipping DDL.                                        ]8;id=939813;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3951893949.py\3951893949.py]8;;\:]8;id=338914;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3951893949.py#15\15]8;;\

           [INFO] Writing data to final path with format=delta...                                  ]8;id=189252;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3951893949.py\3951893949.py]8;;\:]8;id=751905;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3951893949.py#43\43]8;;\

           [INFO]  7/10 Write prerequisites complete (+1.10s, total 8.85s)                            ]8;id=872188;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=516546;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

25/12/01 15:06:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/12/01 15:06:26 WARN MapPartitionsRDD: RDD 98 was locally checkpointed, its lineage has been truncated and cannot be recomputed after unpersisting


[15:06:27] [INFO]  8/10 Write complete (+6.49s, total 15.34s)                                         ]8;id=644090;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=136025;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

           [INFO] Running post-write validations on persisted data...                              ]8;id=485561;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3951893949.py\3951893949.py]8;;\:]8;id=324068;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3951893949.py#79\79]8;;\

[15:06:28] [INFO] Post-write validations passed.                                                   ]8;id=641746;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3951893949.py\3951893949.py]8;;\:]8;id=447273;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/3951893949.py#90\90]8;;\

           [INFO]  9/10 Post-write validations passed (+0.89s, total 16.22s)                          ]8;id=113796;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=980298;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\

## Stop session

Shut down the session once the job completes.


In [9]:
log.log("[INFO] Stopping Spark session.")
spark.stop()
log.log("[INFO] Spark session stopped.")
log_progress("Spark session stopped")


           [INFO] Stopping Spark session.                                                           ]8;id=604935;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/2609902046.py\2609902046.py]8;;\:]8;id=385873;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/2609902046.py#1\1]8;;\

[15:06:29] [INFO] Spark session stopped.                                                            ]8;id=123833;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/2609902046.py\2609902046.py]8;;\:]8;id=730405;file:///var/folders/gc/x7zs83sd6_vcbd1p6t67kj8c0000gn/T/ipykernel_17020/2609902046.py#3\3]8;;\

           [INFO]  10/10 Spark session stopped (+0.69s, total 16.91s)                                 ]8;id=114480;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py\logging.py]8;;\:]8;id=899340;file:///Users/kevin/Github/spark-fuse/src/spark_fuse/utils/logging.py#54\54]8;;\